In [175]:
import pandas as pd

df = pd.read_csv("/home/kyrylo/bucket-72907_mount/football-files/VideoActionTimeStamps.csv")
df

,VidInfo,Action,StartFrame,EndFrame
0,vid1,pass,347,370
1,vid1,goal,583,654
2,vid1,pass,1051,1096
3,vid1,pass,1910,1944
4,vid1,pass,2748,2796
...,...,...,...,...
2552,yt3,pass,67343,67428
2553,yt3,pass,67568,67623
2554,yt3,pass,67950,67977
2555,yt3,pass,70392,70432


In [176]:
df.drop(df[df.Action == 'assist'].index, axis=0, inplace=True)
df

,VidInfo,Action,StartFrame,EndFrame
0,vid1,pass,347,370
1,vid1,goal,583,654
2,vid1,pass,1051,1096
3,vid1,pass,1910,1944
4,vid1,pass,2748,2796
...,...,...,...,...
2552,yt3,pass,67343,67428
2553,yt3,pass,67568,67623
2554,yt3,pass,67950,67977
2555,yt3,pass,70392,70432


In [177]:
df['Len'] = df.EndFrame - df.StartFrame
df

,VidInfo,Action,StartFrame,EndFrame,Len
0,vid1,pass,347,370,23
1,vid1,goal,583,654,71
2,vid1,pass,1051,1096,45
3,vid1,pass,1910,1944,34
4,vid1,pass,2748,2796,48
...,...,...,...,...,...
2552,yt3,pass,67343,67428,85
2553,yt3,pass,67568,67623,55
2554,yt3,pass,67950,67977,27
2555,yt3,pass,70392,70432,40


In [178]:
df.groupby(['Action']).Len.mean()

Action
goal      48.385714
pass      54.620246
save      36.398734
tackle    29.012048
Name: Len, dtype: float64

In [179]:
df.groupby(['Action']).Len.max()

Action
goal      158
pass      210
save      155
tackle    100
Name: Len, dtype: int64

In [180]:
# len(df[df.Len > 100]) -- 120
# df.Action.value_counts() # passes 1500 - tackles x6

In [ ]:
rows = []
for vid in df.VidInfo.unique():
    counter = 0
    for i in range(len(df)):
        if i < len(df) - 1:
            if (df.iloc[i+1].StartFrame - df.iloc[i].EndFrame > df[df.Action == df.iloc[i].Action].Len.mean() + 11) & (df.iloc[i+1].VidInfo == df.iloc[i].VidInfo):
                mean = df[df.Action == df.iloc[i].Action].Len.mean()
                vidname = df.iloc[i].VidInfo
                startframe = df.iloc[i].EndFrame+5
                endframe = startframe+round(mean)

                row = [vidname, 'none', startframe, endframe, endframe-startframe]
                rows.append(row)
                if df.iloc[i+1].StartFrame - endframe > mean + 4:
                    startframe = endframe + 2
                    endframe = startframe + round(mean)
                    row = [vidname, 'none', startframe, endframe, endframe-startframe]
                    rows.append(row)
            counter += 1
            if counter >= 155:
                break
len(rows)

In [183]:
df = pd.concat([df, pd.DataFrame(rows, columns=df.columns)], ignore_index=True)
len(df)

4239

In [190]:
df.drop(df[df.Len > 100].index, inplace=True)
len(df)

4119

In [192]:
df.Action.value_counts()

Action
pass      1760
none      1755
tackle     249
goal       201
save       154
Name: count, dtype: int64

In [201]:
import numpy as np

remove_pass = 260
remove_none = 255

drop_pass = list(np.random.choice(df[df.Action == 'pass'].index, remove_pass, replace=False))
drop_none = list(np.random.choice(df[df.Action == 'none'].index, remove_none, replace=False))

# print(len(drop_none.extend(drop_pass)))
for el in drop_none:
    drop_pass.append(el)

df.drop(drop_pass, inplace=True)
df.Action.value_counts()

Action
pass      1500
none      1500
tackle     249
goal       201
save       154
Name: count, dtype: int64

In [207]:
df

,VidInfo,Action,StartFrame,EndFrame,Len
1,vid1,goal,583,654,71
2,vid1,pass,1051,1096,45
3,vid1,pass,1910,1944,34
4,vid1,pass,2748,2796,48
5,vid1,save,2972,2998,26
...,...,...,...,...,...
4234,vid2,none,27848,27903,55
4235,vid2,none,31987,32016,29
4236,vid2,none,32018,32047,29
4237,vid2,none,35228,35283,55


In [208]:
df.to_csv("/home/kyrylo/dev/actions.csv")